In [1]:
%load_ext autoreload
%autoreload 2
from amt.config import load_model_config
from amt.model import AmtEncoderDecoder, ModelConfig
from aria.tokenizer import AbsTokenizer
import torch
from safetensors.torch import save_file

from amt.model import ModelConfig
from aria.utils import _load_weight
from amt.config import load_model_config
from aria.tokenizer import SeparatedAbsTokenizer

import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os
import torch
import torch.optim

from tqdm import tqdm
from aria.data.midi import MidiDict
from aria.tokenizer import AbsTokenizer

from src.config import get_config
from src.model import P2QTransformer
import os
from functools import partial
from multiprocessing import Pool
from progress.bar import Bar  # Assuming you're using the progress package for the progress bar
from src.dataset import get_ds

from src.load_aria_weights import get_p2q

In [2]:
config = get_config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AbsTokenizer()
train_dataloader, val_dataloader, vocab_size, tokenizer = get_ds(config, train_size_limit=160, batch_size=16)


/home/ubuntu/maikyon/miniconda3/envs/p2q-transformer/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Longest Song Length:  1024
Number of data points in training set:  1294
Number of data points in validation set:  135
Training with batch size:  16
Steps per epoch:  80


In [7]:
# This is all slightly messy as train_loop and val_loop make use of the
# variables in the wider scope. Perhaps refactor this at some point.

def train_loop(dataloader, _epoch: int):
    loss_buffer = []
    model.train()
    for step, batch in (
        pbar := tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            leave=False,
        )
    ):
        enc_input, src, tgt, idxs = batch
        enc_input = enc_input.to(device)
        src = src.to(device)
        tgt = tgt.to(device)
        logits = model(enc_input, src)  # (b_sz, s_len, v_sz)
        logits = logits.transpose(1, 2)  # Transpose for CrossEntropyLoss
        loss = loss_fn(logits, tgt)
        

        writer.add_scalar('train loss', loss.item(), global_step)
        writer.flush()

        loss.backward()
        optimizer.step()

        # Logging
        loss_buffer.append(loss)
        avg_val_loss = sum(loss_buffer) / len(loss_buffer)
        pbar.set_postfix_str(f"average_loss={round(avg_val_loss.item(), 4)}")

    return avg_val_loss

@torch.no_grad()
def val_loop(dataloader, _epoch: int):
    loss_buffer = []
    model.eval()
    for step, batch in (
        pbar := tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            leave=False,
        )
    ):
        enc_input, src, tgt, idxs = batch
        enc_input = enc_input.to(device)
        src = src.to(device)
        tgt = tgt.to(device)
        logits = model(enc_input, src)  # (b_sz, s_len, v_sz)
        logits = logits.transpose(1, 2)  # Transpose for CrossEntropyLoss
        loss = loss_fn(logits, tgt)

        # Logging
        loss_buffer.append(loss)
        avg_val_loss = sum(loss_buffer) / len(loss_buffer)
        pbar.set_postfix_str(f"average_loss={round(avg_val_loss.item(), 4)}")

    return avg_val_loss

In [8]:
model = get_p2q(tokenizer).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], eps=1e-9)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.encode([('<P>')]), label_smoothing=0.1).to(device)

initial_epoch = 0
global_step = 0
writer = SummaryWriter()

for epoch in range(initial_epoch, config['num_epochs']):
    torch.cuda.empty_cache()
    model.train()

    train_loop(train_dataloader, epoch)
    val_loop(val_dataloader, epoch)


    # if epoch % 10 == 0:
    #     run_validation(model, val_dataloader, tokenizer, 512, device, lambda msg: batch_iterator.write(msg), global_step, writer)

KeyboardInterrupt: 